In [1]:
import os
import cv2
import csv
import pandas as pd
import numpy as np
from constants import DatasetPath
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm

In [18]:
DATASET_PATH = DatasetPath.effectivePath

REAL_NAME_DIR = ["afhq", "celebahq", "coco", "ffhq", "imagenet", "landscape", "lsun", "metfaces", "cycle_gan"]

FAKE_NAME_DIR = ["big_gan", "cips", "cycle_gan", "ddpm", "denoising_diffusion_gan", "diffusion_gan", "face_synthetics", 
                 "gansformer", "gau_gan", "generative_inpainting", "glide", "lama", "latent_diffusion", "mat", "palette", 
                 "pro_gan", "projected_gan", "sfhq", "stable_diffusion", "star_gan", "stylegan1", "stylegan2", "stylegan3",
                 "taming_transformer", "vq_diffusion"]

REAL_NAME_DIR_FILTERED = ["coco"]
FAKE_NAME_DIR_FILTERED = ["big_gan"]

DIR_DATASET = os.listdir(DATASET_PATH)

DIR_FOURIER_PATH = DATASET_PATH + "fourier\\"
IMG_FOURIER_PATH = DIR_FOURIER_PATH + "fourier\\"
CSV_REAL_PATH = DATASET_PATH + "real.csv"
CSV_FAKE_PATH = DATASET_PATH + "fake.csv"
TINY_DATASET = DATASET_PATH + "tinydataset.csv"
INPUT_CSV_PATH = DATASET_PATH + "input.csv"

csv_columns_name = ['filename', 'image_path', 'target', 'category']

In [3]:
for folder in DIR_DATASET:
    if folder not in REAL_NAME_DIR + FAKE_NAME_DIR:
        print("Folder "+folder+" does not exist")

Folder fake.csv does not exist
Folder Fourier does not exist
Folder real.csv does not exist
Folder tinydataset.csv does not exist


In [4]:
current_dir = os.listdir(DATASET_PATH)

def create_real_csv():
    if "real.csv" in current_dir:
        print("real.csv already exists.")
        return
    
    real_csv = pd.DataFrame(columns = csv_columns_name)
    #real_csv.to_csv(CSV_REAL_PATH, index=False)

    for dir in REAL_NAME_DIR:
        with open(DATASET_PATH + dir + "\\metadata.csv", mode='r', newline='') as current_csv:

            current_csv = pd.read_csv(current_csv)
            image_path = current_csv["image_path"]
            image_path = dir + "/" + image_path
            image_path = image_path.replace("/","\\")
            current_csv["image_path"] = image_path
            real_csv = pd.concat([real_csv,current_csv[current_csv['target'] == 0]], ignore_index=True)

    real_csv.to_csv(CSV_REAL_PATH, index=False)
    print("real.csv has been created.")

def create_fake_csv():
    if "fake.csv" in current_dir:
        print("fake.csv already exists.")
        return
    
    fake_csv = pd.DataFrame(columns = csv_columns_name)
    #fake_csv.to_csv(CSV_FAKE_PATH, index=False)

    for dir in FAKE_NAME_DIR:
        with open(DATASET_PATH + dir + "\\metadata.csv", mode='r', newline='') as current_csv:

            current_csv = pd.read_csv(current_csv)
            image_path = current_csv["image_path"]
            image_path = dir + "/" + image_path
            image_path = image_path.replace("/","\\")
            current_csv["image_path"] = image_path
            fake_csv = pd.concat([fake_csv,current_csv[current_csv['target'] != 0]], ignore_index=True)

    fake_csv.to_csv(CSV_FAKE_PATH, index=False)
    print("fake.csv has been created.")
            

In [5]:
create_real_csv()
create_fake_csv()

real.csv already exists.
fake.csv already exists.


In [90]:
dir = "big_gan" 

tiny_csv = pd.DataFrame(columns = csv_columns_name)
#tiny_csv.to_csv(TINY_DATASET, index=False)

with open(DATASET_PATH + dir + "\\metadata.csv", mode='r', newline='') as biggan_csv:
    biggan_csv = pd.read_csv(biggan_csv)
    image_path = biggan_csv["image_path"]
    image_path = dir + "/" + image_path
    image_path = image_path.replace("/","\\")
    biggan_csv["image_path"] = image_path
    tiny_csv = pd.concat([tiny_csv,biggan_csv], ignore_index=True)

    tiny_csv.to_csv(TINY_DATASET, index=False)

with open(DATASET_PATH + "coco" + "\\metadata.csv", mode='r', newline='') as coco_csv:
    coco_csv = pd.read_csv(coco_csv)
    image_path = coco_csv["image_path"]
    image_path = "coco" + "/" + image_path
    image_path = image_path.replace("/","\\")
    coco_csv["image_path"] = image_path
    tiny_csv = pd.concat([tiny_csv,coco_csv.sample(n=10000)], ignore_index=True)
    
    tiny_csv.to_csv(TINY_DATASET, index=False)

In [78]:
DIR_DATASET = os.listdir(DATASET_PATH)

if("fourier" in DIR_DATASET):
    print("Fourier folder already exist.")
else:
    mode = 0o666
    path = os.path.join(DATASET_PATH,"fourier")
    os.mkdir(path,mode)
    os.mkdir(path + "\\fourier", mode)

In [86]:
#APPLICO LA TRASFORMATA DI FOURIER SUL TINY DATASET

fourier_csv = pd.DataFrame(columns = csv_columns_name)
with open(DATASET_PATH  + "\\tinydataset.csv", mode='r', newline='') as tiny_dataset:
    tiny_dataset_csv = pd.read_csv(tiny_dataset)
    
    for index, row in tqdm(tiny_dataset_csv.iterrows()):
        path = DATASET_PATH + row["image_path"]
    
        RGBimg = cv2.imread(path)
        grayImg = cv2.cvtColor(RGBimg, cv2.COLOR_BGR2GRAY)
        
        fft_img = np.fft.fft2(grayImg)
        fft_img = np.log(np.abs(fft_img))

        min_val = np.min(fft_img)
        max_val = np.max(fft_img)
        
        #fft_img = cv2.normalize(fft_img, None, 0, 255, cv2.NORM_MINMAX)
        fft_img = (fft_img - min_val) * (255.0 / (max_val - min_val))
        fft_img = np.uint8(fft_img)     
        
        filename = row["filename"]
        if row["target"] == 0:
            filename = filename.replace("img","real")
        else:
            filename = filename.replace("img","fake")

        fft_path = IMG_FOURIER_PATH + filename
        cv2.imwrite(fft_path, fft_img)
        
        split_fft_path = fft_path.split("ArtiFact dataset\\")[-1]
        fourier_csv.loc[len(fourier_csv)] = [filename,split_fft_path,row["target"],row["category"]]
    
    fourier_csv.to_csv(DIR_FOURIER_PATH + "metadata.csv", index=False)
    
    #plt.imshow(fft_img)
    #plt.show()

0it [00:00, ?it/s]

In [ ]:
img = cv2.imread("C:\\Users\\mario\\Desktop\\Biometria\\ArtiFact dataset\\fourier\\fourier\\fake000000.jpg", -1)
print(img.shape)
cv2.imshow("img",img)
cv2.waitKey(0)
plt.imshow(img)
plt.show()

In [ ]:
input_df = pd.DataFrame(columns= ["anchor","positive","negative"])

with open(DIR_FOURIER_PATH + "metadata.csv", mode='r', newline='') as fourier_csv:
    fourier_df = pd.read_csv(fourier_csv)
    input_df["anchor"] = fourier_df["image_path"]

    real_fourier_df = fourier_df[ fourier_df["target"] == 0 ]
    fake_fourier_df = fourier_df[ fourier_df["target"] != 0 ]
    
    print(real_fourier_df.head())
    for index, row in input_df.iterrows():
        image_path = row["anchor"]

        if "real" in image_path:
            positive_img = real_fourier_df.sample(frac=1).head(1)
            while positive_img["image_path"].iloc[0] == image_path:
                positive_img = real_fourier_df.sample(frac=1).head(1)
            input_df.at[index, "positive"] = positive_img["image_path"].iloc[0]
            negative_img = fake_fourier_df.sample(frac=1).head(1)
            input_df.at[index, "negative"] = negative_img["image_path"].iloc[0]
        elif "fake" in image_path:
            positive_img = real_fourier_df.sample(frac=1).head(1)
            input_df.at[index, "negative"] = positive_img["image_path"].iloc[0]
            negative_img = fake_fourier_df.sample(frac=1).head(1)
            while negative_img["image_path"].iloc[0] == image_path:
                negative_img = fake_fourier_df.sample(frac=1).head(1)
            input_df.at[index, "positive"] = negative_img["image_path"].iloc[0]
            

input_df.to_csv(DATASET_PATH + "input.csv", index=False)
    